<a href="https://colab.research.google.com/github/mb8655/Python/blob/main/Inserting_Data_in_MySQL_using_PythonPandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inserting and Reading data from MySQL using Pandas
First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [41]:
!pip install -U -q PyMySQL sqlalchemy

import requests
import pandas as pd
import sqlalchemy

In [42]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = "https://gbfs.citibikenyc.com/gbfs/en/station_status.json"

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json()
results

{'data': {'stations': [{'region_id': '71',
    'eightd_station_services': [],
    'external_id': '1817822909864556798',
    'capacity': 27,
    'station_type': 'classic',
    'eightd_has_key_dispenser': False,
    'lon': -73.91274864321404,
    'station_id': '1817822909864556798',
    'short_name': '7082.08',
    'electric_bike_surcharge_waiver': False,
    'rental_methods': ['KEY', 'CREDITCARD'],
    'rental_uris': {'android': 'https://bkn.lft.to/lastmile_qr_scan',
     'ios': 'https://bkn.lft.to/lastmile_qr_scan'},
    'lat': 40.77423301703027,
    'has_kiosk': True,
    'name': '23 Ave & 31 St'},
   {'region_id': '71',
    'eightd_station_services': [],
    'external_id': '823ef531-40e6-4397-b973-86bf5b032076',
    'capacity': 24,
    'station_type': 'classic',
    'eightd_has_key_dispenser': False,
    'lon': -73.93679,
    'station_id': '823ef531-40e6-4397-b973-86bf5b032076',
    'short_name': '3862.07',
    'electric_bike_surcharge_waiver': False,
    'rental_methods': ['KEY', 'C

In [43]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]
len(data)



2233

In [44]:
df = pd.DataFrame(data)
df.head(5)

,region_id,eightd_station_services,external_id,capacity,station_type,eightd_has_key_dispenser,lon,station_id,short_name,electric_bike_surcharge_waiver,rental_methods,rental_uris,lat,has_kiosk,name
0,71,[],1817822909864556798,27,classic,False,-73.912749,1817822909864556798,7082.08,False,"[KEY, CREDITCARD]",{'android': 'https://bkn.lft.to/lastmile_qr_sc...,40.774233,True,23 Ave & 31 St
1,71,[],823ef531-40e6-4397-b973-86bf5b032076,24,classic,False,-73.936790,823ef531-40e6-4397-b973-86bf5b032076,3862.07,False,"[KEY, CREDITCARD]",{'android': 'https://bkn.lft.to/lastmile_qr_sc...,40.669290,True,Eastern Pkwy & Troy Ave
2,71,[],b4974d89-932c-4a9f-a264-2ea0cf70613e,20,classic,False,-73.937878,b4974d89-932c-4a9f-a264-2ea0cf70613e,8458.03,False,"[KEY, CREDITCARD]",{'android': 'https://bkn.lft.to/lastmile_qr_sc...,40.855331,True,Pinehurst Ave & W 187 St
3,71,[],1786700154621508184,29,classic,False,-73.943950,1786700154621508184,5656.03,False,"[KEY, CREDITCARD]",{'android': 'https://bkn.lft.to/lastmile_qr_sc...,40.727700,True,N Henry St & Norman Ave
4,71,[],66db6963-0aca-11e7-82f6-3863bb44ef7c,78,classic,False,-73.986923,66db6963-0aca-11e7-82f6-3863bb44ef7c,5863.07,False,"[KEY, CREDITCARD]",{'android': 'https://bkn.lft.to/lastmile_qr_sc...,40.734232,True,E 15 St & 3 Ave


In [45]:
# We drop the 'rental methods' columns,
# as they contains multiple values and
# we cannot insert lists in a database cell.
df.drop(
    ["rental_methods", "eightd_station_services", "rental_uris"],
    axis="columns",
    inplace=True,
)
df.head(5)

,region_id,external_id,capacity,station_type,eightd_has_key_dispenser,lon,station_id,short_name,electric_bike_surcharge_waiver,lat,has_kiosk,name
0,71,1817822909864556798,27,classic,False,-73.912749,1817822909864556798,7082.08,False,40.774233,True,23 Ave & 31 St
1,71,823ef531-40e6-4397-b973-86bf5b032076,24,classic,False,-73.936790,823ef531-40e6-4397-b973-86bf5b032076,3862.07,False,40.669290,True,Eastern Pkwy & Troy Ave
2,71,b4974d89-932c-4a9f-a264-2ea0cf70613e,20,classic,False,-73.937878,b4974d89-932c-4a9f-a264-2ea0cf70613e,8458.03,False,40.855331,True,Pinehurst Ave & W 187 St
3,71,1786700154621508184,29,classic,False,-73.943950,1786700154621508184,5656.03,False,40.727700,True,N Henry St & Norman Ave
4,71,66db6963-0aca-11e7-82f6-3863bb44ef7c,78,classic,False,-73.986923,66db6963-0aca-11e7-82f6-3863bb44ef7c,5863.07,False,40.734232,True,E 15 St & 3 Ave


##Writing a Pandas Dataframe in a MySQL Table


In [46]:
from sqlalchemy import create_engine, text

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2025"
)

engine = create_engine(conn_string)

In [47]:
# Query to create a database
# In this example, we will try to create the (existing) database "public"
# But in general, we can give any name to the database
db_name = "public"
create_db_query = (
    f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"
)

# Create a database
with engine.connect() as con:
    con.execute(text(create_db_query))


# And lets switch to the database
with engine.connect() as con:
    con.execute(text(f"USE {db_name}"))

In [48]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid

if "suffix" not in globals():
  suffix = str(uuid.uuid4())[:8]
print(suffix)

3e20d832


Create Table and Store Data in Database using the to_sql command
Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [49]:
drop_table_sql = f'''
DROP TABLE IF EXISTS {db_name}.{table_name};
'''
table_name = f"Stations_{suffix}"
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation

# This step is optional, but it is good practice to define explicitly the
# data types before storing things in a database. In many cases, this can be ommitted, though.

create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name}
                                (legacy_id int,
                                station_id varchar(100),
                                region_id int,
                                external_id varchar(50),
                                lat float,
                                lon float,
                                short_name varchar(10),
                                name varchar(250),
                                station_type varchar(10),
                                capacity int,
                                electric_bike_surcharge_waiver bool,
                                eightd_has_key_dispenser bool,
                                has_kiosk bool,
                                PRIMARY KEY(station_id)
                                )"""

with engine.connect() as con:
  con.execute(text(create_table_query))

In [50]:
sql = f"""SELECT * FROM {db_name}.{table_name}"""

with engine.connect() as con:
    df2 = pd.read_sql(sql.format(db_name=db_name, table_name=table_name), con=con)
df2

,legacy_id,station_id,region_id,external_id,lat,lon,short_name,name,station_type,capacity,electric_bike_surcharge_waiver,eightd_has_key_dispenser,has_kiosk
0,None,00284700-9d22-42ce-8485-113fed9879c1,71.0,00284700-9d22-42ce-8485-113fed9879c1,40.7641,-73.9107,6879.04,28 Ave & 44 St,classic,19,0,0,1
1,None,002e6e9f-ced2-47b0-bc7e-ccf38c3b4b0f,71.0,002e6e9f-ced2-47b0-bc7e-ccf38c3b4b0f,40.8142,-73.9039,7830.03,Prospect Ave & E 151 St,classic,21,0,0,1
2,None,00967b8f-1a4d-4131-a65a-17fa8ca89e28,71.0,00967b8f-1a4d-4131-a65a-17fa8ca89e28,40.7445,-73.8976,6225.03,41 Ave & 67 St,classic,20,0,0,1
3,None,0096fecb-3a56-4585-9204-aa9554f7bc49,71.0,0096fecb-3a56-4585-9204-aa9554f7bc49,40.7248,-73.9843,5626.06,E 5 St & Ave A,classic,48,0,0,1
4,None,00d3e65e-b614-4408-aaa9-939ed96ce645,71.0,00d3e65e-b614-4408-aaa9-939ed96ce645,40.7359,-74.0009,5955.12,Perry St & Greenwich Ave,classic,53,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,None,ff9581d3-3e1a-463a-bdab-d8150c468be2,71.0,ff9581d3-3e1a-463a-bdab-d8150c468be2,40.7746,-73.9237,7126.01,21 St & Hoyt Ave S,classic,33,0,0,1
2229,None,ffaa8f6a-a4ea-4d3a-a57c-f42f77cd4581,71.0,ffaa8f6a-a4ea-4d3a-a57c-f42f77cd4581,40.6785,-73.9137,4138.04,Thomas S. Boyland St & Fulton St,classic,24,0,0,1
2230,None,ffae66ec-7c16-436f-bd0a-eedf81d580e7,71.0,ffae66ec-7c16-436f-bd0a-eedf81d580e7,40.8766,-73.8837,8778.01,E Mosholu Pkwy & Van Cortlandt Ave E,classic,22,0,0,1
2231,None,ffef95fa-600b-4f08-b2ce-e8eb034ebfd8,71.0,ffef95fa-600b-4f08-b2ce-e8eb034ebfd8,40.7435,-73.9186,6209.08,46 St & Queens Blvd,classic,33,0,0,1


In [51]:
df.to_sql(
    name=table_name,
    schema=db_name,
    con=engine,
    if_exists="append",
    index=False
)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1817822909864556798' for key 'Stations_3e20d832.PRIMARY'")
[SQL: INSERT INTO public.`Stations_3e20d832` (region_id, external_id, capacity, station_type, eightd_has_key_dispenser, lon, station_id, short_name, electric_bike_surcharge_waiver, lat, has_kiosk, name) VALUES (%(region_id)s, %(external_id)s, %(capacity)s, %(station_type)s, %(eightd_has_key_dispenser)s, %(lon)s, %(station_id)s, %(short_name)s, %(electric_bike_surcharge_waiver)s, %(lat)s, %(has_kiosk)s, %(name)s)]
[parameters: [{'region_id': '71', 'external_id': '1817822909864556798', 'capacity': 27, 'station_type': 'classic', 'eightd_has_key_dispenser': 0, 'lon': -73.91274864321404, 'station_id': '1817822909864556798', 'short_name': '7082.08', 'electric_bike_surcharge_waiver': 0, 'lat': 40.77423301703027, 'has_kiosk': 1, 'name': '23 Ave & 31 St'}, {'region_id': '71', 'external_id': '823ef531-40e6-4397-b973-86bf5b032076', 'capacity': 24, 'station_type': 'class

In [ ]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way

# add_key_query = f"ALTER TABLE {table_name} ADD PRIMARY KEY(station_id)"
# print(add_key_query)

# with engine.connect() as con:
#  con.execute(text(add_key_query))

#Reading from a SQL Database in Python using the read_sql command in Pandas
We can similarly read from the database using Pandas

In [52]:
query = text(f"SELECT * FROM {db_name}.{table_name}")
print(query)

SELECT * FROM public.Stations_3e20d832


In [53]:
with engine.connect() as con:
  df2 = pd.read_sql(query, con=con)
df2.head(5)

,legacy_id,station_id,region_id,external_id,lat,lon,short_name,name,station_type,capacity,electric_bike_surcharge_waiver,eightd_has_key_dispenser,has_kiosk
0,None,00284700-9d22-42ce-8485-113fed9879c1,71.0,00284700-9d22-42ce-8485-113fed9879c1,40.7641,-73.9107,6879.04,28 Ave & 44 St,classic,19,0,0,1
1,None,002e6e9f-ced2-47b0-bc7e-ccf38c3b4b0f,71.0,002e6e9f-ced2-47b0-bc7e-ccf38c3b4b0f,40.8142,-73.9039,7830.03,Prospect Ave & E 151 St,classic,21,0,0,1
2,None,00967b8f-1a4d-4131-a65a-17fa8ca89e28,71.0,00967b8f-1a4d-4131-a65a-17fa8ca89e28,40.7445,-73.8976,6225.03,41 Ave & 67 St,classic,20,0,0,1
3,None,0096fecb-3a56-4585-9204-aa9554f7bc49,71.0,0096fecb-3a56-4585-9204-aa9554f7bc49,40.7248,-73.9843,5626.06,E 5 St & Ave A,classic,48,0,0,1
4,None,00d3e65e-b614-4408-aaa9-939ed96ce645,71.0,00d3e65e-b614-4408-aaa9-939ed96ce645,40.7359,-74.0009,5955.12,Perry St & Greenwich Ave,classic,53,0,0,1


#Export Data from Database to CSV or Excel
And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [54]:
# The necessary library to write in Excel
!sudo pip3 install -U openpyxl

In [56]:
df2.to_excel("citibike.xlsx")
df2.to_csv("citibike.csv")

#Cleanup
Finally, let's clean up and delete the table that we created

In [57]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
print(drop_table_query)

with engine.connect() as con:
  con.execute(text(drop_table_query))

DROP TABLE IF EXISTS public.Stations_3e20d832
